##### This is my training project to learn LangGraph agents, the idea is to create an Agent with different tools, then ask it different things and check if is using the tools and the process.


In [ ]:
%pip install -U langgraph 

In [5]:
# Load variables
from dotenv import load_dotenv 
load_dotenv()
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

In [2]:
# Define the model
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [6]:
response = model.invoke([{"role": "user", "content": 'Hi there!'}])
print(response.content)

Hi! How can I help you today?
